In [1]:
import pickle
import warnings
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from gc import collect
from scipy.optimize import basinhopping


warnings.filterwarnings("ignore", message="X does not have valid feature names, but IncrementalPCA was fitted with feature names")
pd.options.mode.chained_assignment = None

In [3]:
full = pd.read_csv('data/test_set_VU_DM.csv', parse_dates=['date_time'])
enc = np.load('data/enc_test.npy')

full = full.drop(columns=['site_id', 'visitor_location_country_id', 'prop_country_id'])
full = split_time(fillnans(full))
full.head()

,srch_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,...,comp3_rate_missing,comp4_rate_missing,comp5_rate_missing,comp6_rate_missing,comp7_rate_missing,comp8_rate_missing,year,month,day,time
0,1,-1.0,-1.0,3180,3,4.5,1,2.94,0.0691,5.03,...,1,1,1,1,1,1,13,2,2,0.644213
1,1,-1.0,-1.0,5543,3,4.5,1,2.64,0.0843,4.93,...,1,1,1,1,1,1,13,2,2,0.644213
2,1,-1.0,-1.0,14142,2,3.5,1,2.71,0.0556,4.16,...,1,1,0,1,1,1,13,2,2,0.644213
3,1,-1.0,-1.0,22393,3,4.5,1,2.40,0.0561,5.03,...,1,1,0,1,1,1,13,2,2,0.644213
4,1,-1.0,-1.0,24194,3,4.5,1,2.94,0.2090,4.72,...,1,1,0,1,1,1,13,2,2,0.644213


## Predicting

## Ranking

In [2]:
full = pd.read_csv('data/test_set_VU_DM.csv', usecols=['prop_id'])
df = pd.read_csv('data/comparisons.csv', index_col=False)
df.head()

,srch,first,second,greater,lesser
0,1,1,0,0.421361,0.563120
1,1,2,0,0.554154,0.426303
2,1,3,0,0.627818,0.351510
3,1,4,0,0.516800,0.452182
4,1,5,0,0.374522,0.619263


In [3]:
df['score'] = (df.greater + 1 - df.lesser) / 2
df['score'] = np.log(df.score / (1 - df.score))
df['score'] = df.score.replace(np.infty, 35).replace(-np.infty, -35)

C:\Users\Hackerman\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
def evaluate(x, scores):
    pairs = [(int(a), int(b)) for i, a in enumerate(x) for b in x[i+1:]]
    return -scores.loc[pairs].sum()

def permutation(x):
    if np.random.uniform() < 0.2:
        return np.random.permutation(x)
    flip = np.random.choice(range(len(x)), 2, replace=False)
    x[flip] = x[flip[::-1]]
    return x

In [210]:
def swap(x, swaps, scores):
    seqs, seq = [], []
    seq = swaps[:1]
    for i in swaps[1:]:
        if seq[-1] + 1 == i:
            seq.append(i)
        else:
            seqs.append(seq)
            seq = [i]
    seqs.append(seq)
    for s in seqs:
        if len(s) == 1:
            x[[s[0], s[0]+1]] = x[[s[0]+1, s[0]]]
        else:
            evals = []
            for i in range(len(s)):
                t = x[s[0]:s[-1]+2].copy()
                t[[i, i+1]] = t[[i+1, i]]
                evals.append(evaluate(t, scores))
            i = s[evals.index(max(evals))]
            x[[i, i+1]] = x[[i+1, i]]
    return x
            

def resort(x, scores):
    change = True
    while change:
        change = False
        swaps = list()
        for i, (a, b) in enumerate(zip(x[:-1], x[1:])):
            if scores[a, b] > 0:
                swaps.append(i)
                change = True
        if change:
            x = swap(x, swaps, scores)
    return x

In [217]:
open('ans.csv', 'w').write('srch_id,prop_id\n')
for srch, x in tqdm(df.groupby('srch')):
    scores = x.set_index(['first', 'second']).score
    scores = pd.concat([scores, -scores.swaplevel(0, 1)])
    x = scores.unstack().sum(axis=1).sort_values().index.values.flatten()
    x = resort(x, scores)
    # res = basinhopping(lambda x: evaluate(x, scores), rank,
    #                    take_step=permutation,
    #                    niter=1600, niter_success=200)
    pd.DataFrame({'srch_id': srch, 'prop_id': full.loc[x].values.flatten()}).to_csv('ans.csv', mode='a', index=False, header=False)

100%|█████████████████████████████████████████████████████████████████████████| 199549/199549 [31:43<00:00, 104.85it/s]
